# Data Wrangling

## Preparacion de datos

### Carga de datos

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
# Load the dataset
df_clean_companies = pd.read_csv('../data/raw/sp500_companies.csv')
df_clean_index = pd.read_csv('../data/raw/sp500_index.csv')
df_clean_stocks = pd.read_csv('../data/raw/sp500_stocks.csv')

### Preparacion de los datos df_raw_companies

In [151]:
df_clean_companies.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,254.49,3846819807232,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.069209
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,134.70,3298803056640,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.059350
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,436.60,3246068596736,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058401
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,224.92,2365033807872,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042550
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,191.41,2351625142272,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042309


#### Limpieza

##### Valores duplicados

In [152]:
# Verificar duplicados 
duplicados = df_clean_companies.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")

Número de filas duplicadas: 0


##### Valores unicos

In [153]:
# Verificar el número de valores únicos por columna
valores_unicos = df_clean_companies.nunique()
print("Número de valores únicos por columna:")
print(valores_unicos)

Número de valores únicos por columna:
Exchange                 4
Symbol                 502
Shortname              499
Longname               499
Sector                  11
Industry               114
Currentprice           499
Marketcap              502
Ebitda                 469
Revenuegrowth          260
City                   235
State                   41
Country                  8
Fulltimeemployees      387
Longbusinesssummary    499
Weight                 502
dtype: int64


In [154]:
# Verificar los valores únicos de una columna ej Sector
valores_unicos_sector = df_clean_companies['Sector'].unique()
print("Valores únicos en la columna:")
print(valores_unicos_sector)

Valores únicos en la columna:
['Technology' 'Consumer Cyclical' 'Communication Services'
 'Financial Services' 'Consumer Defensive' 'Healthcare' 'Energy'
 'Basic Materials' 'Industrials' 'Utilities' 'Real Estate']


##### Valores nulos

In [155]:
# Verificar valores nulos
valores_nulos = df_clean_companies.isnull().sum()
print("Valores nulos por columna:")
valores_nulos = valores_nulos[valores_nulos > 0]
print(valores_nulos)

# Porcentaje de valores nulos por columna
porcentaje_nulos = (df_clean_companies.isnull().sum() / len(df_clean_companies)) * 100
print("\nPorcentaje de valores nulos por columna:")
porcentaje_nulos = porcentaje_nulos[porcentaje_nulos > 0]
print(porcentaje_nulos)

Valores nulos por columna:
Ebitda               29
Revenuegrowth         3
State                20
Fulltimeemployees     9
dtype: int64

Porcentaje de valores nulos por columna:
Ebitda               5.776892
Revenuegrowth        0.597610
State                3.984064
Fulltimeemployees    1.792829
dtype: float64


Al ver que el porcentaje de valores nulos es bajo, podemos toomar las siguientes acciones para cada variable:
- Ebitda: imputar valores con la mediana
- Revenuegrowth: imputar valores basados en el crecimiento promedio de la industria
- State: se puede tratar como una categoria separada "Desconocido"
- Fulltimeemployees: imputar valores con la media del sector


In [156]:
# 'Ebitda' con la mediana
df_clean_companies['Ebitda'].fillna(df_clean_companies['Ebitda'].median(), inplace=True)

# 'Revenuegrowth' con el promedio del industria
df_clean_companies['Revenuegrowth'] = df_clean_companies.groupby('Industry')['Revenuegrowth'].transform(
    lambda x: x.fillna(x.mean())
)

# 'State' con la categoría "Desconocido"
df_clean_companies['State'].fillna('Desconocido', inplace=True)

# 'Fulltimeemployees' con la media del sector
df_clean_companies['Fulltimeemployees'] = df_clean_companies.groupby('Sector')['Fulltimeemployees'].transform(
    lambda x: x.fillna(x.mean())
)

C:\Users\gabyz\AppData\Local\Temp\ipykernel_13224\1729402439.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean_companies['Ebitda'].fillna(df_clean_companies['Ebitda'].median(), inplace=True)
C:\Users\gabyz\AppData\Local\Temp\ipykernel_13224\1729402439.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [157]:
# Verificar valores nulos
valores_nulos = df_clean_companies.isnull().sum()
print("Valores nulos por columna:")
valores_nulos = valores_nulos[valores_nulos > 0]
print(valores_nulos)

Valores nulos por columna:
Series([], dtype: int64)


#### Transformacion

In [158]:
# Transformacion de datos (Fulltimeemployees, Marketcap) a int 
df_clean_companies['Fulltimeemployees'] = df_clean_companies['Fulltimeemployees'].astype('int')
df_clean_companies['Marketcap'] = df_clean_companies['Marketcap'].astype('int')

# Verificar los cambios
print(df_clean_companies.dtypes)

Exchange                object
Symbol                  object
Shortname               object
Longname                object
Sector                  object
Industry                object
Currentprice           float64
Marketcap                int64
Ebitda                 float64
Revenuegrowth          float64
City                    object
State                   object
Country                 object
Fulltimeemployees        int64
Longbusinesssummary     object
Weight                 float64
dtype: object


### Preparacion de los datos df_raw_index

In [159]:
df_clean_index.head()

,Date,S&P500
0,2014-12-22,2078.54
1,2014-12-23,2082.17
2,2014-12-24,2081.88
3,2014-12-26,2088.77
4,2014-12-29,2090.57


#### Limpieza

##### Valores duplicados

In [160]:
# Verificar duplicados
duplicados = df_clean_index.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")

Número de filas duplicadas: 0


##### Valores nulos

In [161]:
# Verificar valores nulos en el DataFrame
valores_nulos = df_clean_index.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos)

# Porcentaje de valores nulos por columna
porcentaje_nulos = (df_clean_index.isnull().sum() / len(df_clean_index)) * 100
print("\nPorcentaje de valores nulos por columna:")
print(porcentaje_nulos)

Valores nulos por columna:
Date      0
S&P500    0
dtype: int64

Porcentaje de valores nulos por columna:
Date      0.0
S&P500    0.0
dtype: float64


No hay valores nulos ni duplicados

#### Transformacion

Transformamos la variable date a datetime

In [162]:
# 'Date' to datetime
df_clean_index['Date'] = pd.to_datetime(df_clean_index['Date'])

In [163]:
df_clean_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2517 non-null   datetime64[ns]
 1   S&P500  2517 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 39.5 KB


Creamos variables derivadas para posibles analisis por periodos separados

In [164]:
# Variables derivadas dia, mes, año
df_clean_index['Year'] = df_clean_index['Date'].dt.year  
df_clean_index['Month'] = df_clean_index['Date'].dt.month  
df_clean_index['Day'] = df_clean_index['Date'].dt.day  

# Verificar las nuevas columnas
df_clean_index.head()

,Date,S&P500,Year,Month,Day
0,2014-12-22,2078.54,2014,12,22
1,2014-12-23,2082.17,2014,12,23
2,2014-12-24,2081.88,2014,12,24
3,2014-12-26,2088.77,2014,12,26
4,2014-12-29,2090.57,2014,12,29


### Preparacion de los datos df_raw_stocks

In [24]:
df_clean_stocks.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04,MMM,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-05,MMM,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-06,MMM,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-07,MMM,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-08,MMM,NaN,NaN,NaN,NaN,NaN,NaN


#### Limpieza

##### Valores duplicados

In [166]:
# Verificar duplicados
duplicados = df_clean_stocks.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")

Número de filas duplicadas: 0


##### Valores nulos

In [167]:
# Verificar valores nulos
valores_nulos = df_clean_stocks.isnull().sum()
print("Valores nulos por columna:")
valores_nulos = valores_nulos[valores_nulos > 0]
print(valores_nulos)

# Porcentaje de valores nulos por columna
porcentaje_nulos = (df_clean_stocks.isnull().sum() / len(df_clean_stocks)) * 100
print("\nPorcentaje de valores nulos por columna:")
porcentaje_nulos = porcentaje_nulos[porcentaje_nulos > 0]
print(porcentaje_nulos)

Valores nulos por columna:
Adj Close    1273705
Close        1273705
High         1273705
Low          1273705
Open         1273705
Volume       1273705
dtype: int64

Porcentaje de valores nulos por columna:
Adj Close    67.337074
Close        67.337074
High         67.337074
Low          67.337074
Open         67.337074
Volume       67.337074
dtype: float64


En este caso el porcentaje de valores nulos es bastante alto (67,33%) y dada la importancia de las columnas, podriamos optar por imputar valores.Se debe tomar en cuenta que la desviacion estandar es muy alta por lo que rellenar con la media puede afectar el analisis, por lo que se puede considerar rellenar con valores de dias cercanos.




In [28]:
# Imprimir filas con valores NaN
filas_con_nan = df_clean_stocks[df_clean_stocks.isnull().any(axis=1)]
print(filas_con_nan)

               Date Symbol  Adj Close  Close  High  Low  Open  Volume
0        2010-01-04    MMM        NaN    NaN   NaN  NaN   NaN     NaN
1        2010-01-05    MMM        NaN    NaN   NaN  NaN   NaN     NaN
2        2010-01-06    MMM        NaN    NaN   NaN  NaN   NaN     NaN
3        2010-01-07    MMM        NaN    NaN   NaN  NaN   NaN     NaN
4        2010-01-08    MMM        NaN    NaN   NaN  NaN   NaN     NaN
...             ...    ...        ...    ...   ...  ...   ...     ...
1891531  2024-12-16    ZTS        NaN    NaN   NaN  NaN   NaN     NaN
1891532  2024-12-17    ZTS        NaN    NaN   NaN  NaN   NaN     NaN
1891533  2024-12-18    ZTS        NaN    NaN   NaN  NaN   NaN     NaN
1891534  2024-12-19    ZTS        NaN    NaN   NaN  NaN   NaN     NaN
1891535  2024-12-20    ZTS        NaN    NaN   NaN  NaN   NaN     NaN

[1273705 rows x 8 columns]


In [31]:
# Valores cercanos
df_clean_stocks['Adj Close'].fillna(method='ffill', inplace=True)  # Usar el valor posterior
df_clean_stocks['Close'].fillna(method='bfill', inplace=True)  
df_clean_stocks['High'].fillna(method='ffill', inplace=True)  # Usar el valor anterior
df_clean_stocks['Low'].fillna(method='bfill', inplace=True)  
df_clean_stocks['Open'].fillna(method='ffill', inplace=True)  
df_clean_stocks['Volume'].fillna(method='bfill', inplace=True) 

C:\Users\gabyz\AppData\Local\Temp\ipykernel_1280\4029708530.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean_stocks['Adj Close'].fillna(method='ffill', inplace=True)  # Usar el valor posterior
C:\Users\gabyz\AppData\Local\Temp\ipykernel_1280\4029708530.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean_stocks['Adj Close'].fillna(method='ffill', inplace=True)  # Usar el valor posterior
C:\Users\gabyz\AppData\Local\Temp

Otro caso que podriamo evaluar es utilizar modelos de ML para predecir los valores faltantes segun otras columnas. Este caso no se realizara pero dejo el posible codigo para realizar este paso.

In [169]:
#from sklearn.impute import KNNImputer
#
#imputer = KNNImputer(n_neighbors=5)
#df_clean_stocks[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']] = imputer.fit_transform(
#    df_clean_stocks[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']]
#)

In [170]:
# Verificar valores nulos
valores_nulos = df_clean_stocks.isnull().sum()
print("Valores nulos por columna:")
valores_nulos = valores_nulos[valores_nulos > 0]
print(valores_nulos)

# Porcentaje de valores nulos por columna
porcentaje_nulos = (df_clean_stocks.isnull().sum() / len(df_clean_stocks)) * 100
print("\nPorcentaje de valores nulos por columna:")
porcentaje_nulos = porcentaje_nulos[porcentaje_nulos > 0]
print(porcentaje_nulos)

Valores nulos por columna:
Adj Close     3768
Close        15072
High          3768
Low          15072
Open          3768
Volume       15072
dtype: int64

Porcentaje de valores nulos por columna:
Adj Close    0.199203
Close        0.796813
High         0.199203
Low          0.796813
Open         0.199203
Volume       0.796813
dtype: float64


Se rellenaron las instancias con los valores cercanos, sin embargo siguen exitiendo valores nulos. En este caso el porcentaje es bajo, por lo que se puede considerar eliminar estas filas.

In [171]:
# Eliminar filas con valores nulos
df_clean_stocks = df_clean_stocks.dropna()

# Verificar el número de filas después de eliminar las filas nulas
print(f"Filas restantes: {len(df_clean_stocks)}")


Filas restantes: 1872696


In [172]:
# Verificar valores nulos
valores_nulos = df_clean_stocks.isnull().sum()
print("Valores nulos por columna:")
valores_nulos = valores_nulos[valores_nulos > 0]
print(valores_nulos)


Valores nulos por columna:
Series([], dtype: int64)


#### Transformacion

In [176]:
# 'Date' to datetime
df_clean_stocks['Date'] = pd.to_datetime(df_clean_stocks['Date'])

# Volume to int
df_clean_stocks['Volume'] = df_clean_stocks['Volume'].astype('int')


# Verificar los cambios
print(df_clean_stocks.dtypes)

Date         datetime64[ns]
Symbol               object
Adj Close           float64
Close               float64
High                float64
Low                 float64
Open                float64
Volume                int64
Year                  int32
Month                 int32
Day                   int32
dtype: object


Creamos variables derivadas para posibles analisis por periodos separados

In [174]:
# Variables derivadas dia, mes, año
df_clean_stocks['Year'] = df_clean_stocks['Date'].dt.year  
df_clean_stocks['Month'] = df_clean_stocks['Date'].dt.month  
df_clean_stocks['Day'] = df_clean_stocks['Date'].dt.day  

# Verificar las nuevas columnas
df_clean_stocks.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Year,Month,Day
3768,2010-01-04,AOS,5.937266,7.435000,7.480000,7.261667,7.295000,1104600,2010,1,4
3769,2010-01-05,AOS,5.861404,7.340000,7.431667,7.308333,7.431667,1207200,2010,1,5
3770,2010-01-06,AOS,5.864068,7.343333,7.405000,7.301667,7.335000,663000,2010,1,6
3771,2010-01-07,AOS,5.881369,7.365000,7.425000,7.311667,7.356667,564000,2010,1,7
3772,2010-01-08,AOS,5.967879,7.473333,7.485000,7.311667,7.331667,504600,2010,1,8


### Guardar Data

In [178]:
dataframes = {'df_clean_companies': df_clean_companies, 'df_clean_index': df_clean_index, 'df_clean_stocks': df_clean_stocks}

for name, df in dataframes.items():
    file_path = f"../data/clean/{name}.csv"
    df.to_csv(file_path, index=False)
    print(f"Archivo guardado: {file_path}")

Archivo guardado: ../data/clean/df_clean_companies.csv
Archivo guardado: ../data/clean/df_clean_index.csv
Archivo guardado: ../data/clean/df_clean_stocks.csv
